In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px

In [13]:
# atrinkti 10 populiariausių gamintojų iš 
# autopliuslt, pateikti tokius stulpelius - gamintojas, vidutinė kaina, vidutinė rida, 
# vidutinis amžius, modelių kiekis. Gautus rezultatus surašyti į SQLite duomebazę, 
# pavadinimu "Testas.db", lentelės pavadinimas kokį sugalvosite. Viską turi atlikti python kodas. 
# PANDAS NEGALIMA NAUDOTI, viską atlikite per cursor() objektus.

In [14]:
#prisijungimas prie mySQL, ir kursoriaus sukūrimas
DB = cnt.connect(
    host = '88.223.121.130',
    user='user',
    password='slaptazodis',
    port=3306,
    database='works'
    )
C=DB.cursor()


SDB = sqlite3.connect('Testas.db')
SC = SDB.cursor()

In [15]:
#  lentelės pasiėmimas:
#sql užklausos vykdymas per mysql/mariadb kursorių
sql="""
select G, cast(avg(P) as float), cast(avg(R) as float), cast(avg(A) as float), count(*) as C from
(select gamintojas as G, cast(replace(price, ' ','') as int) as P,
cast(replace(rida, ' ','') as int) as R, 
2024 - cast(substring(pagaminimo_data, 1,4) as int) as A
from autopliuslt) as T1
group by G
order by C desc limit 10;
"""
C.execute(sql)
ans = C.fetchall()
# ? = list(map(list, zip(*ans)))
#? = np.transpose(ans)

print(ans)

[('BMW', 10652.1, 260654.0, 15.2219, 365), ('Volkswagen', 6190.71, 237651.0, 15.7903, 310), ('Audi', 8447.67, 241376.0, 16.7665, 227), ('Mercedes-Benz', 15080.3, 261259.0, 15.1864, 177), ('Toyota', 8712.73, 201755.0, 14.6039, 154), ('Volvo', 7847.71, 278037.0, 15.3116, 138), ('Opel', 4005.84, 226788.0, 16.3209, 134), ('Ford', 5592.4, 225685.0, 14.8818, 110), ('Peugeot', 6748.5, 205881.0, 13.4103, 78), ('Skoda', 10808.3, 181196.0, 10.7703, 74)]


In [16]:
sql_table = '''
create table if not exists TAuto
(
    G text not null,
    P real not null,
    R real not null,
    A real not null,
    C real not null
    
)
'''
SC.execute(sql_table)

sql_template = '''insert into TAuto values (?,?,?,?,?)'''

SC.executemany(sql_template, ans)

SDB.commit()
SDB.close()

In [ ]:
# pandas ir JSON:

#  https://data.gov.lt/datasets/509/

# nuskaitykite JSON failą -  2023 metus

# konvertuokite dataLaikas stulpelį į datetime
#  pd.to_datetime(df[stulpelis])

# pridėkite naują stulpelį su paros laikais (rytas, vakaras ...)
# pridėkite naują stulplelį, kur būtų tik įvykio valanda
# Vizualizuokite, kuriuo paros laiku, kurią valandą įvyksta daugiausiai eismo įvykių

# sekantiems klausimams patys sugalvojate tinkamiausias vizualizacijas
# Vyrų ar moterų buvo daugiau, sukėlusių eismo įvykį? (stulpelis eismoEalyviai)
#  koks vidutinis kaltininkų amžius?
# Kokiomis baudų rūšimis buvo nubausti vyrai/moterys (administracinė, baudžiamoji) (teisenosStadijaBusena)

In [21]:
df = pd.read_json('../DATA/ei_2023_12_31.json')
df.head(1)

,registrokodas,dataLaikas,registravimoData,paskutinioRedagavimoLaikas,iskaitinis,rusis,schema1,schema2,dalyviuSkaicius,zuvusiuSkaicius,...,kelioElementas2,privalomasLeistinasGreitis,leistinasGreitis,neblaivusKaltininkai,apsvaigeKaltininkai,atsisakeTikrintisKaltininkai,ilguma,platuma,eismoDalyviai,eismoTranspPreimone
0,07-23-9588945,2023-01-01 00:27,2023-01-01 01:04,2023-03-31 12:14,0,Kiti eismo įvykiai,Kiti eismo įvykiai,Kiti eismo įvykiai,2,0,...,None,1,40.0,Ne,Ne,Ne,6063990.0,579292.0,"[{'dalyvisId': 48690423, 'kategorija': 'Kiti',...","[{'tpId': 33646065, 'regValstybe': 'Lietuva', ..."


In [22]:
df['dataLaikas'] = pd.to_datetime(df['dataLaikas'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24240 entries, 0 to 24239
Data columns (total 52 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   registrokodas                 24240 non-null  object        
 1   dataLaikas                    24240 non-null  datetime64[ns]
 2   registravimoData              24240 non-null  object        
 3   paskutinioRedagavimoLaikas    24240 non-null  object        
 4   iskaitinis                    24240 non-null  int64         
 5   rusis                         24240 non-null  object        
 6   schema1                       24240 non-null  object        
 7   schema2                       24240 non-null  object        
 8   dalyviuSkaicius               24240 non-null  int64         
 9   zuvusiuSkaicius               24240 non-null  int64         
 10  zuvVaiku                      24240 non-null  int64         
 11  suzeistuSkaicius            